### Load sample log.

Sample log is csv with three columns, (1) date, (2) userID, (3)CampaignID  - like following.
```
click.at    user.id campaign.id
2015/4/27 20:40 144012  Campaign077
2015/4/27 0:27  24485   Campaign063
2015/4/27 0:28  24485   Campaign063
2015/4/27 0:33  24485   Campaign038
```

In [1]:
import json, os, datetime, collections, commands
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *

if not os.path.exists("./click_data_sample.csv"):
    print "csv file not found at master node, will download and copy to HDFS"
    commands.getoutput("wget -q http://image.gihyo.co.jp/assets/files/book/2015/978-4-7741-7631-4/download/click_data_sample.csv")
    commands.getoutput("hadoop fs -copyFromLocal -f ./click_data_sample.csv /user/hadoop/")

whole_raw_log = sc.textFile("/user/hadoop/click_data_sample.csv")
header = whole_raw_log.first()
whole_log = whole_raw_log.filter(lambda x:x !=header).map(lambda line: line.split(","))\
            .map(lambda line: [datetime.datetime.strptime(line[0].replace('"', ''), '%Y-%m-%d %H:%M:%S'), int(line[1]), line[2].replace('"', '')])

whole_log.take(3)

[[datetime.datetime(2015, 4, 27, 20, 40, 40), 144012, u'Campaign077'],
 [datetime.datetime(2015, 4, 27, 0, 27, 55), 24485, u'Campaign063'],
 [datetime.datetime(2015, 4, 27, 0, 28, 13), 24485, u'Campaign063']]

### Create Spark Dataframe
* can create from RDD with providing schema info. [Refer this page for shema definition](http://spark.apache.org/docs/latest/sql-programming-guide.html#data-types)
* `StructField('date', TimestampType(), True)` 's last argument (True or False) indicate "null" is allowed or not
* dataframe can be cached with same manner as RDD by `dataframe.repartition(4).cache()`

In [2]:
fields = [StructField('access_time', TimestampType(), True), StructField('userID', IntegerType(), True), StructField('campaignID', StringType(), True)]
schema = StructType(fields)

whole_log_df = sqlContext.createDataFrame(whole_log, schema)
print whole_log_df.printSchema()
print whole_log_df.dtypes
print whole_log_df.show(10)

root
 |-- access_time: timestamp (nullable = true)
 |-- userID: integer (nullable = true)
 |-- campaignID: string (nullable = true)

None
[('access_time', 'timestamp'), ('userID', 'int'), ('campaignID', 'string')]
+--------------------+------+-----------+
|         access_time|userID| campaignID|
+--------------------+------+-----------+
|2015-04-27 20:40:...|144012|Campaign077|
|2015-04-27 00:27:...| 24485|Campaign063|
|2015-04-27 00:28:...| 24485|Campaign063|
|2015-04-27 00:33:...| 24485|Campaign038|
|2015-04-27 01:00:...| 24485|Campaign063|
|2015-04-27 16:10:...|145066|Campaign103|
|2015-04-27 20:06:...|145066|Campaign103|
|2015-04-27 14:52:...|167405|Campaign027|
|2015-04-27 22:08:...|167405|Campaign027|
|2015-04-27 20:11:...| 80524|Campaign054|
+--------------------+------+-----------+
only showing top 10 rows

None


### Usage for query with SQL expression
* To use SQL expression, first need to set table name by `registerTempTable`
* You can perform sub queries, but you need to add `alias` to sub query - otherwise it fails.
  * OK : `SELECT count(*) FROM (SELECT * FROM my_table limit 10) subquery_alias`
  * fails : `SELECT count(*) FROM (SELECT * FROM my_table limit 10)`

In [3]:
#Simple SQL query

whole_log_df.registerTempTable("whole_log_table")

print sqlContext.sql(" SELECT * FROM whole_log_table where campaignID == 'Campaign047' ").count()
print sqlContext.sql(" SELECT * FROM whole_log_table where campaignID == 'Campaign047' ").show(5)

#SQL query with variables
for count in range(1, 3):
    print "Campaign00" + str(count)
    print sqlContext.sql("SELECT count(*) as access_num FROM whole_log_table where campaignID == 'Campaign00" + str(count) + "'").show()

#Sub-query : noted you must add alias to subquery otherwise it fails.
print sqlContext.sql(" SELECT count(*) as first_count FROM (SELECT userID, min(access_time) as first_access_date FROM whole_log_table GROUP BY userID) subquery_alias WHERE first_access_date < '2015-04-28'").show(5)

18081
+--------------------+------+-----------+
|         access_time|userID| campaignID|
+--------------------+------+-----------+
|2015-04-27 05:26:...| 14151|Campaign047|
|2015-04-27 05:26:...| 14151|Campaign047|
|2015-04-27 05:26:...| 14151|Campaign047|
|2015-04-27 05:27:...| 14151|Campaign047|
|2015-04-27 05:28:...| 14151|Campaign047|
+--------------------+------+-----------+
only showing top 5 rows

None
Campaign001
+----------+
|access_num|
+----------+
|      2407|
+----------+

None
Campaign002
+----------+
|access_num|
+----------+
|      1674|
+----------+

None
+-----------+
|first_count|
+-----------+
|      20480|
+-----------+

None


### Usage for `fitler` and `select`
* `filter` extract rows with certain condition
* `select` extract columns

In [4]:
#Sample for filter

print whole_log_df.filter(whole_log_df["access_time"] < "2015-04-28").count()
print whole_log_df.filter(whole_log_df["access_time"] > "2015-05-01").show(5)

41434
+--------------------+------+-----------+
|         access_time|userID| campaignID|
+--------------------+------+-----------+
|2015-05-01 22:11:...|114157|Campaign002|
|2015-05-01 23:36:...| 93708|Campaign055|
|2015-05-01 22:51:...| 57798|Campaign046|
|2015-05-01 21:21:...|  1437|Campaign019|
|2015-05-01 00:19:...| 24485|Campaign011|
+--------------------+------+-----------+
only showing top 5 rows

None


In [5]:
#Sample for select

print whole_log_df.select("access_time", "userID").show(5)

+--------------------+------+
|         access_time|userID|
+--------------------+------+
|2015-04-27 20:40:...|144012|
|2015-04-27 00:27:...| 24485|
|2015-04-27 00:28:...| 24485|
|2015-04-27 00:33:...| 24485|
|2015-04-27 01:00:...| 24485|
+--------------------+------+
only showing top 5 rows

None


### Usage for `groupBy + agg`
1. example of **groupBy** which is similar of **reduceByKey**
2. example of **groupBy + agg**
  * argument of agg should be `agg({key:value})` : `key` to be colum name, `value` to be function name(like `min`,`sum`, `ave` etc) 
  * method of `agg` only works for the output of `groupBy`=(GroupedData Ojb), and not work for normal dataframe object

In [6]:
#Example(1) simple groubBy -> count(). You can set multiple keys like groupBy('key1', 'key2'). 
print whole_log_df.groupBy('campaignID').count().sort('count', ascending=False).show(5)
print whole_log_df.groupBy('campaignID', 'userID').count().sort('count', ascending=False).show(5)

+-----------+-----+
| campaignID|count|
+-----------+-----+
|Campaign116|22193|
|Campaign027|19206|
|Campaign047|18081|
|Campaign107|13295|
|Campaign131| 9068|
+-----------+-----+
only showing top 5 rows

None
+-----------+------+-----+
| campaignID|userID|count|
+-----------+------+-----+
|Campaign047| 30292|  633|
|Campaign086|107624|  623|
|Campaign047|121150|  517|
|Campaign086| 22975|  491|
|Campaign122| 90714|  431|
+-----------+------+-----+
only showing top 5 rows

None


In [7]:
#Example(2)
print whole_log_df.groupBy('userID').agg({"access_time": "min"}).show(3)
print whole_log_df.groupBy('userID').agg({"access_time": "min"}).printSchema()
print whole_log_df.groupBy('userID').agg({"access_time": "min"}).filter("min(access_time) < '2015-04-28'").count()

+------+--------------------+
|userID|    min(access_time)|
+------+--------------------+
|  4831|2015-04-27 22:49:...|
| 48631|2015-04-27 22:15:...|
|143031|2015-04-27 21:52:...|
+------+--------------------+
only showing top 3 rows

None
root
 |-- userID: integer (nullable = true)
 |-- min(access_time): timestamp (nullable = true)

None
20480


### UDF(User Defined Function)
* Spark dataframe support UDF and typical usage will add new column (or replace existing) with UDF.

In [8]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DoubleType

def add_day_column(access_time):
    return int(access_time.strftime('%Y%m%d'))
    
my_udf = UserDefinedFunction(add_day_column, IntegerType())
print whole_log_df.withColumn("access_day", my_udf("access_time")).show(10)

+--------------------+------+-----------+----------+
|         access_time|userID| campaignID|access_day|
+--------------------+------+-----------+----------+
|2015-04-27 20:40:...|144012|Campaign077|  20150427|
|2015-04-27 00:27:...| 24485|Campaign063|  20150427|
|2015-04-27 00:28:...| 24485|Campaign063|  20150427|
|2015-04-27 00:33:...| 24485|Campaign038|  20150427|
|2015-04-27 01:00:...| 24485|Campaign063|  20150427|
|2015-04-27 16:10:...|145066|Campaign103|  20150427|
|2015-04-27 20:06:...|145066|Campaign103|  20150427|
|2015-04-27 14:52:...|167405|Campaign027|  20150427|
|2015-04-27 22:08:...|167405|Campaign027|  20150427|
|2015-04-27 20:11:...| 80524|Campaign054|  20150427|
+--------------------+------+-----------+----------+
only showing top 10 rows

None


### Convert from dataframe to another object
* convert to RDD : `map` will convert from dataframe to RDD
* convert to (normal) list : first convert to RDD (by `map()`), then convert to list (by `collect()`)

In [9]:
#convert to rdd
print whole_log_df.groupBy('campaignID').count().map(lambda x: [x[0], x[1]]).take(5)

[[u'Campaign033', 786], [u'Campaign034', 3867], [u'Campaign035', 963], [u'Campaign036', 1267], [u'Campaign037', 1010]]


In [10]:
#convert to list
print whole_log_df.groupBy('campaignID').count().map(lambda x: [x[0], x[1]]).collect()[:5]
print len(whole_log_df.groupBy('campaignID').count().map(lambda x: [x[0], x[1]]).collect())

[[u'Campaign033', 786], [u'Campaign034', 3867], [u'Campaign035', 963], [u'Campaign036', 1267], [u'Campaign037', 1010]]
133


### Other

In [11]:
# columns will returns list of column strings
print whole_log_df.columns

['access_time', 'userID', 'campaignID']
